In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [40]:
input_str='apple'
label_str='pple!'
char_vocab=sorted(list(set(input_str+label_str))) #set으로 중복제거, 리스트담아서, sorted
# applepple! => [a, p, l, e, !] => [!, a, e, l, p]
vocab_size=len(char_vocab)
print('문자집합:',char_vocab)
print('문자집합의크기:',vocab_size)

문자집합: ['!', 'a', 'e', 'l', 'p']
문자집합의크기: 5


In [41]:
input_size=5
hidden_size=5
output_size=5
learning_rate=0.1

In [42]:
index_to_char=dict((i,c) for i, c in enumerate(char_vocab))
print(index_to_char)

char_to_index=dict((c,i) for i, c in enumerate(char_vocab))
print(char_to_index)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}
{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [43]:
# 훈련데이터생성
x_data=[char_to_index[c] for c in input_str]
y_data=[char_to_index[c] for c in label_str]
x_data=[x_data]
y_data=[y_data]
print(x_data)
print(y_data)

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


원핫인코딩
x값을 인코딩한 것

In [44]:
# x_dataone hot encoding
x_one_hot=[np.eye(vocab_size)[x] for x in x_data] 
#eye는 대각행렬, 데이터 x갑에 1넣기
#대각행렬은 현재 주어지는 x값이 1을 넣어라.
#원핫인코딩으로 a는 1하고 나머지는 0, vocar_size는 크기가 5, x값에 해당하는 부분은 1세팅
print(x_one_hot) #리스트가 2번이라서 2차원데이터가 나와

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


대각행렬 eye

In [45]:
np.eye(3)[0] #3은 크기, 0 위치에 1
np.eye(3)[1] #3은 크기, 2 위치에 1

array([0., 1., 0.])

In [46]:
# 훈련 데이터 텐서로 변환
X=torch.FloatTensor(x_one_hot) #는 풀롯
Y=torch.LongTensor(y_data) #y는 분류를 해야하는 수라서 long 텐서로 나와야해
print(X, Y)

print(X.size(), Y.size()) #x는 3차원 y는 2차원으로 만들어짐

tensor([[[0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 1., 0.],
         [0., 0., 1., 0., 0.]]]) tensor([[4, 4, 3, 2, 0]])
torch.Size([1, 5, 5]) torch.Size([1, 5])


In [47]:
# 모델정의클래스
class Net(nn.Module): #Net이든 RNN이든
  def __init__(self, input_size, hidden_size, output_size):
    super(Net, self).__init__()
    self.rnn=nn.RNN(input_size, hidden_size, batch_first=True)
    #input은 몇개, batch_first=True 입력의 차원의 수를 어떻게 해석할지 지정하는 옵션
    #입력차원을 (batch_size, sql_len, input_size) 배치사이즈를 첫번째로 보겠다.

    self.fc=nn.Linear(hidden_size, output_size, bias=True) #끝에 분류를 해야해, 출력은 리니어,

  def forward(self, x):
      x, _status=self.rnn(x)
      x=self.fc(x) #리니어를 넣고
      return x


In [48]:
# 모델생성, 비용함수/ 최적화설정
net=Net(input_size, hidden_size, output_size) #net으로 모델만들어
crit=nn.CrossEntropyLoss() #손실함수는 여러개라서 크로스엔트로피
optimizer=optim.Adam(net.parameters(), lr=learning_rate) #lr 생략가능

In [49]:
output=net(X) #파이토치 텐서라서 처음에 출력하면 날것의 결과
print(output)
print(output.argmax(-1))
print(output.shape) # apple -> pple! 44320 - 4 4 4 0 2 # 나와

tensor([[[-0.8674, -0.2123, -0.3341,  0.2397,  1.1486],
         [-0.4943, -0.1562,  0.3079,  0.3758,  0.8329],
         [-0.7495, -0.0577,  0.1122,  0.0027,  0.9738],
         [-0.7833, -0.0577, -0.0745,  0.0686,  1.0268],
         [-0.6057, -0.2588, -0.0098,  0.2929,  0.7590]]],
       grad_fn=<ViewBackward0>)
tensor([[4, 4, 4, 4, 4]])
torch.Size([1, 5, 5])


In [50]:
print(output.view(-1, input_size))
print(Y.view(-1))

tensor([[-0.8674, -0.2123, -0.3341,  0.2397,  1.1486],
        [-0.4943, -0.1562,  0.3079,  0.3758,  0.8329],
        [-0.7495, -0.0577,  0.1122,  0.0027,  0.9738],
        [-0.7833, -0.0577, -0.0745,  0.0686,  1.0268],
        [-0.6057, -0.2588, -0.0098,  0.2929,  0.7590]],
       grad_fn=<ViewBackward0>)
tensor([4, 4, 3, 2, 0])


In [51]:
result=output.data.numpy().argmax(axis=2)
print(result)

[[4 4 4 4 4]]


In [52]:
result1=[index_to_char[i] for i in np.squeeze(result)]
print(result1)

str_result=''.join([index_to_char[i] for i in np.squeeze(result)]) #문자열 조인 공부!!
#2차원 -> 1차원 -> 리스트 -> join
print(str_result)

['p', 'p', 'p', 'p', 'p']
ppppp


In [53]:
# learing
for epoch in range(100):
  optimizer.zero_grad()
  output=net(X)
  
  loss=crit(output.view(-1, input_size), Y.view(-1))  #인풋사이즈만큼 모양을 바꿔서 뷰값
  #아웃풋크기를 맞춰 (-1, input_size), view(-1) 차원맞춰?
  loss.backward()
  optimizer.step() 

  result=output.data.numpy().argmax(axis=2) #아그맥스해서 숫자 높은 값 골라
  str_result=''.join([index_to_char[i] for i in np.squeeze(result)]) #인덱스 넣으면 캐릭터값나와
  #2차원을 1차원으로 고치고, squeeze, 숫자로 나온 c 값과 조인을 해주면 문자로 나와
  print(f'{epoch}, loss:{loss.item()}, prediction:{result},\
        true Y:{y_data}, prediction_str:{str_result}')
  print(result)

0, loss:1.566527009010315, prediction:[[4 4 4 4 4]],        true Y:[[4, 4, 3, 2, 0]], prediction_str:ppppp
[[4 4 4 4 4]]
1, loss:1.3353006839752197, prediction:[[4 3 3 2 3]],        true Y:[[4, 4, 3, 2, 0]], prediction_str:pllel
[[4 3 3 2 3]]
2, loss:1.1165008544921875, prediction:[[4 3 3 2 2]],        true Y:[[4, 4, 3, 2, 0]], prediction_str:pllee
[[4 3 3 2 2]]
3, loss:0.8671040534973145, prediction:[[4 4 3 2 0]],        true Y:[[4, 4, 3, 2, 0]], prediction_str:pple!
[[4 4 3 2 0]]
4, loss:0.6212301850318909, prediction:[[4 4 3 2 0]],        true Y:[[4, 4, 3, 2, 0]], prediction_str:pple!
[[4 4 3 2 0]]
5, loss:0.4457477629184723, prediction:[[4 4 3 2 0]],        true Y:[[4, 4, 3, 2, 0]], prediction_str:pple!
[[4 4 3 2 0]]
6, loss:0.30576345324516296, prediction:[[4 4 3 2 0]],        true Y:[[4, 4, 3, 2, 0]], prediction_str:pple!
[[4 4 3 2 0]]
7, loss:0.21246464550495148, prediction:[[4 4 3 2 0]],        true Y:[[4, 4, 3, 2, 0]], prediction_str:pple!
[[4 4 3 2 0]]
8, loss:0.1565529853105